In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
import os
load_dotenv(r"H:\My_LangGraph_toturial\.env")

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = os.getenv(var)
        print("var" , var)
        print("key" , os.getenv(var))
_set_env("OPENAI_API_KEY")

openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools: list[tool] = [add, multiply, divide]
llm : ChatOpenAI = ChatOpenAI(model = "gpt-3.5-turbo-0125")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph.state import CompiledStateGraph

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

class default_state(MessagesState):
    summary: str = "my simple string"

first_call = True
# Node
def assistant(state: MessagesState):
    global first_call
    if first_call:
        first_call = False
        return {"messages": [llm_with_tools.invoke([sys_msg])] , "summary": "my first string"}
    else:
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder: StateGraph = StateGraph(default_state)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine the control flow
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory: MemorySaver = MemorySaver()
graph: CompiledStateGraph = builder.compile(checkpointer=memory)

# Show
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
from pprint import pprint
import json

# Input
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}

# Thread
thread = {"configurable": {"thread_id": "1"}}
for i, event in enumerate(graph.stream(initial_input, thread, stream_mode="values")):
    print(f"Event {i}:")
    # print("Keys in event:", event.keys())  # Print the keys to understand the structure
    # Optionally, pretty-print the entire event for detailed inspection
    pprint(event)
    # print("Last message:")
    # event['messages'][-1].pretty_print()  # Print the last message nicely
    print("-" * 50)  # Separator for clarity


# # Run the graph until the first interruption
# for event in graph.stream(initial_input, thread, stream_mode="values"):
#     event['messages'][-1].pretty_print()